In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from process_czi import get_channel_info, get_czi_info, _get_chunk_bbox, chunk_and_save_czi, _get_channel_idx
from io_images import get_images_infoframe
import czifile
import os
import aicspylibczi
import numpy as np
import plotly.express as px

In [3]:
# Brain Slices
# folder_path = 'data'
folder_path = "/run/user/1001/gvfs/smb-share:server=fs.ista.ac.at,share=drives/tnegrell/archive/siegegrp/AlVe/MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS"


In [4]:
# List all files and directories in the folder
contents = os.listdir(folder_path)

print("Contents of the folder:")
for item in contents:
    print(item)

Contents of the folder:
siegegrp00037.czi
siegegrp00032.czi
siegegrp00033.czi
siegegrp00034.czi
MAX_C1-siegegrp00030.czi - siegegrp00030.czi #1.tif
siegegrp00035.czi
siegegrp00043.czi
C2-siegegrp00030.czi - siegegrp00030.czi #1.tif
siegegrp00031.czi
siegegrp00040.czi
siegegrp00041.czi
siegegrp00042.czi
siegegrp00036.czi
siegegrp00030.czi
siegegrp00039.czi
siegegrp00038.czi


In [5]:
# infoframe = get_images_infoframe(folder_path, conditions=['Animal'])
infoframe = get_images_infoframe(folder_path)

Found 14 files...
There are 14 files in folder_location


In [6]:
infoframe

,file_name,file_path
0,siegegrp00037.czi,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
1,siegegrp00032.czi,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
2,siegegrp00033.czi,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
3,siegegrp00034.czi,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
4,siegegrp00035.czi,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
5,siegegrp00043.czi,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
6,siegegrp00031.czi,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
7,siegegrp00040.czi,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
8,siegegrp00041.czi,/run/user/1001/gvfs/smb-share:server=fs.ista.a...
9,siegegrp00042.czi,/run/user/1001/gvfs/smb-share:server=fs.ista.a...


In [7]:
czi_path = infoframe.iloc[5]['file_path']
czi_path

'/run/user/1001/gvfs/smb-share:server=fs.ista.ac.at,share=drives/tnegrell/archive/siegegrp/AlVe/MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS/siegegrp00043.czi'

In [8]:
get_channel_info(czi_path)


Channel ID: Channel:0
Name: DAPI
Dye: DAPI
Emission: 465
Excitation: 353
Color: #FF00A0FF
------------------------------
Channel ID: Channel:1
Name: EGFP
Dye: EGFP
Emission: 509
Excitation: 488
Color: #FF00FF5B
------------------------------


In [13]:
czi_file = aicspylibczi.CziFile(czi_path)
# Take the correct chanel that contains the nucleus
metadata = czi_file.meta
correct_channel_idx = _get_channel_idx(metadata, 'DAPI')

bbox = czi_file.get_all_scene_bounding_boxes()
sequences = czi_file.get_dims_shape()
nb_seq = len(sequences)
print(f"There are {nb_seq} tissue slices in this file")
maxproj_list = []

There are 2 tissue slices in this file


In [12]:
from utils import downsample_by_2

In [ ]:
s=0
z_len = sequences[s]['Z'][1] - sequences[s]['Z'][0]  # Depth (Z-axis)
x_min, y_min, w, h = bbox[s].x, bbox[s].y, bbox[s].w, bbox[s].h  # Bounding box details
print(w,h)
previous_image = np.zeros((w, h))
previous_image = downsample_by_2(previous_image)

<class 'int'>


In [17]:
z_image = czi_file.read_mosaic(region = (x_min, y_min, w, h), 
                                            scale_factor = 1.0, 
                                            C = correct_channel_idx, 
                                            Z = 0,
                                            dtype = np.uint8,
                                            )[0][0]

In [21]:
z_image.shape[0]*z_image.shape[1]/1024**3*51*2

66.19188538938761

In [22]:
z_image = downsample_by_2(z_image)


In [24]:
z_image = np.transpose(z_image, (1, 0))


In [25]:
previous_image = np.maximum(z_image, previous_image)


In [26]:
previous_image.shape

(11143, 15633)

In [ ]:
from utils import plot_image
plot_image(previous_image)

In [ ]:

for s, seq in enumerate(sequences):
    if seq['S'][1]-seq['S'][0] == 1:
        z_len = sequences[s]['Z'][1] - sequences[s]['Z'][0]  # Depth (Z-axis)
        x_min, y_min, w, h = bbox[s].x, bbox[s].y, bbox[s].w, bbox[s].h  # Bounding box details
        previous_image = np.zeros(w, h)
        previous_image = downsample_by_2(previous_image)
        for z in range(z_len):
            z_image = czi_file.read_mosaic(region = (x_min, y_min, w, h), 
                                            scale_factor = 1.0, 
                                            C = correct_channel_idx, 
                                            Z = z,
                                            dtype = np.uint8,
                                            )[0][0] # The function returns an array (1,1,y,x)
            z_image = downsample_by_2(z_image)
            z_image = np.transpose(z_image, (1, 0))
            previous_image = np.maximum(z_image, previous_image)
        maxproj_list.append(previous_image)